# Atividade: Perceptron - Petróleo

**INSTITUTO FEDERAL DE MINAS GERAIS**
*Departamento de Engenharia e Computação*

**Professor:** Ciniro Nametala  
**Aluno:** Emanuel Rocha

In [102]:
#importacao de pacotes
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import plotly.graph_objects as go

In [103]:
#funcao degau bipolar
def bipolar(u):
    return np.where(u >= 0, 1, -1)
#funcao pra classifica a amostra com base na rede ja treinada
def previsao(w,x):
  u=np.dot(w.T,x)
  return bipolar(u)

In [104]:
#PASSO 1: OBTER O CONJUNTO DE AMOSTRAS DE TREINAMENTO X
dados = pd.read_csv("tabela_treino.csv", sep=";")
nump=5
#quantidade de elementos na amostra
n_amostras=dados.shape[0]
#quantidade variaveis de entrada(subrair a coluna de tipo)
n_variaveis=dados.shape[1]-1
#separando os dados contendo apenas as variaveis de entrada
x=dados.iloc[:,1:n_variaveis].values
#print(x)
n_variaveis=x.shape[1]
#print(n_variaveis)

In [105]:
#inserindo o bias
bias=np.ones((n_amostras,1))*-1
x=np.hstack((x,bias))
#print(x)

In [106]:
#PASSO 2: DO VETOR DE SAIDAS Y, ASSOCIAR A SAIDA ESPERADA Y PARA CADA X
y=dados.iloc[:,-1].values
#print(y)
#contar quantas amostras de cada classe existem no dataset
#print(f"-1: {(y==-1).sum()}")
#print(f"1: {(y==1).sum()}")

### 1)  Treine uma RNA com arquitetura Perceptron cinco vezes (T1 até T5), sempre iniciando o vetor de pesos w com valores aleatórios entre 0 e 1. Garanta que em cada uma das cinco tentativas de treinamento os pesos iniciais do vetor w sejam diferentes. Garanta que o erro seja baixo testando diferentes números de épocas e observando a convergência do erro médio. Após definir um total de épocas suficiente, mantenha a mesma quantidade de épocas para todos os cinco treinos.

In [107]:
#INICIALIZAR OS VETORES DE PESOS W COM VALORES ALEATORIOS
pesos = [np.random.rand(n_variaveis + 1) for i in range(nump)]
pesos_iniciais=[w.copy() for w in pesos]
#for i in range(nump):
#   print(f"w{i+1}: {pesos[i]}")

In [108]:
#DEFINIR A TAXA DE APRENDIZAGEM COM VALOR ENTRE 0 E 1 E TOLERANCIA
eta=0.01
tolerancia=0.02

#SETAR O ERRO INICIAL E CRIAR O VETOR QUE PRA ARMAZENAR OS ERROS DAS EPOCAS
erro_medio=[tolerancia+1]*nump
erro_epocas = [[] for i in range(nump)]
epocas = [0] * nump
max_epocas=10000

In [109]:
for idx in range(nump):
    erro_medio = tolerancia + 1
    epoca = 0
    w = pesos[idx]

    while erro_medio >= tolerancia and epoca < max_epocas:
        epoca += 1
        erro_atual = 0
        indices = np.random.permutation(n_amostras)

        for i in indices:
            xi = x[i, :]
            u = np.dot(w.T, xi)
            yhat = bipolar(u)
            e = y[i] - yhat
            w += eta * e * xi
            erro_atual += np.abs(e)

        erro_medio = erro_atual / n_amostras
        erro_epocas[idx].append(erro_medio)

    pesos[idx] = w
    epocas[idx] = epoca

In [110]:
#for idx in range(nump):    
#    plt.figure(figsize=(4,4))
#    plt.plot(erro_epocas[idx])
#    plt.title(f'Convergência do erro - Perceptron {idx+1}')
#    plt.xlabel('Épocas')
#    plt.ylabel('Erro médio')
#    plt.grid(True)
#    plt.show()

In [111]:
# Lendo as novas amostras pra classificar
dados_classif = pd.read_csv("tabela_classificacao.csv", sep=";")
x_classif = dados_classif.iloc[:, 1:n_variaveis+1].values
bias_classif = np.ones((x_classif.shape[0], 1)) * -1
x_classif = np.hstack((x_classif, bias_classif))
#print(x_classif)

### 2) Registre os resultados dos pesos antes e depois dos treinos, em cada um dos cinco treinos conforme a Tabela 1:

In [112]:
num_pesos = n_variaveis + 1
cabecalho = "Treino".ljust(8)
for i in range(num_pesos):
    cabecalho += f"w{i}_ini".rjust(10)
for i in range(num_pesos):
    cabecalho += f"w{i}_fim".rjust(10)
print(cabecalho)

for idx in range(nump):
    linha = f"{'Treino '+str(idx+1)}".ljust(8)
    # pesos iniciais
    for w in pesos_iniciais[idx]:
        linha += f"{w:10.4f}"
    # pesos finais
    for w in pesos[idx]:
        linha += f"{w:10.4f}"
    print(linha)

Treino      w0_ini    w1_ini    w2_ini    w3_ini    w0_fim    w1_fim    w2_fim    w3_fim
Treino 1    0.4776    0.6744    0.5422    0.2438    0.3936    0.9455   -0.1735   -0.7562
Treino 2    0.2845    0.1454    0.8351    0.7357    0.3726    0.7839   -0.0934   -0.7843
Treino 3    0.3015    0.2410    0.8604    0.5647    0.3692    0.8872   -0.0399   -0.7353
Treino 4    0.7700    0.0199    0.1737    0.2174    0.3360    0.8703   -0.2954   -0.6626
Treino 5    0.4941    0.4191    0.8865    0.1149    0.3718    0.8818   -0.1834   -0.6051


### 3) Após o treinamento dos modelos, aplique-os separadamente na classificação das amostras de óleo da tabela abaixo (arquivo tabela_classificacao.csv). Anote na Tabela 2 os resultados das saídas (classes) para cada amostra e para cada modelo treinado (T1 até T5).

In [113]:
print("\nTabela de Classificação:")
cabecalho = "Amostra".ljust(10) + "x1".rjust(10) + "x2".rjust(10) + "x3".rjust(10)
for i in range(nump):
    cabecalho += f"T{i+1}".rjust(10)
print(cabecalho)

# coloca cada peso de cada modelo pra cada amostra
for j in range(x_classif.shape[0]):
    # Amostra + valores de entrada
    linha = f"{str(j+1)}".ljust(10)
    for valor in x_classif[j, :n_variaveis]:  # pega só x1, x2, x3 (sem o bias)
        linha += f"{valor:10.4f}"

    # Saídas dos modelos
    for i in range(nump):
        w = pesos[i]
        y_pred = previsao(w, x_classif[j])
        linha += f"{y_pred:10}"
    print(linha)


Tabela de Classificação:
Amostra           x1        x2        x3        T1        T2        T3        T4        T5
1            -0.3665    0.0620    5.9891        -1         1         1        -1        -1
2            -0.7842    1.1267    5.5912         1         1         1        -1         1
3             0.3012    0.5611    5.8234         1         1         1        -1         1
4             0.7757    1.0648    8.0677         1         1         1        -1         1
5             0.1570    0.8028    6.3040         1         1         1        -1         1
6            -0.7014    1.0316    3.6005         1         1         1         1         1
7             0.3748    0.1536    6.1537        -1         1         1        -1        -1
8            -0.6920    0.9404    4.4058         1         1         1        -1         1
9            -1.3970    0.7141    4.9263         1         1         1        -1        -1
10           -1.8842   -0.2805    1.2548        -1        -1    

### 4) Qual foi sua taxa de acerto percentual para cada modelo?

> testei com a propria tabela treino porque a de classificacao não tem os resultados pra poder comparar.


In [114]:
yhat = [np.zeros(n_amostras) for _ in range(nump)]

for idx in range(nump):
    for i in range(n_amostras):
        yhat[idx][i] = previsao(pesos[idx], x[i,:])
for idx in range(nump):
    acuracia = (np.sum(yhat[idx] == y) / n_amostras) * 100
    print(f"Taxa de acerto percentual para o modelo {idx+1}: {acuracia:.2f}%")

Taxa de acerto percentual para o modelo 1: 63.33%
Taxa de acerto percentual para o modelo 2: 46.67%
Taxa de acerto percentual para o modelo 3: 46.67%
Taxa de acerto percentual para o modelo 4: 63.33%
Taxa de acerto percentual para o modelo 5: 66.67%


### 5) Qual o efeito de aumentar ou diminuir o número de épocas na qualidade dos resultados?

> Não tem efeito algum, rodar com 500 ou 1 milhão de epocas no máximo não muda nada, a taxa de acerto continua na mesma faixa e as vezes com os mesmos valores.

### 6) Qual o efeito de aumentar ou diminuir a taxa de aprendizagem na qualidade dos resultados?

> Não tem efeito nenhum na qualidade dos resultados, os modelos sempre ficam acima de 0.2 de erro, mas no resultado o valor não afeta.

### 7) As classes neste problema são linearmente separáveis?

> Não, mesmo plotando o gráfico 3d não e possivel ver como um plano poderia separar corretamente as classes.

In [115]:
#colors = ['red' if yi == -1 else 'blue' for yi in y]
#fig = go.Figure()
#fig.add_trace(go.Scatter3d(
#    x=x[:, 0],
#    y=x[:, 1],
#    z=x[:, 2],
#    mode='markers',
#    marker=dict(
#        size=6,
#        color=colors,
#        opacity=0.8,
#        line=dict(width=1, color='white')
#    ),
#    name='Amostras'
#))
#fig.update_layout(
#    title='Distribuição das Amostras no Espaço 3D',
#    scene=dict(
#        xaxis_title='x1',
#        yaxis_title='x2',
#        zaxis_title='x3'
#    ),
#    width=800,
#    height=600
#)
#fig.show()